In [1]:
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import shutil
import time 

In [2]:
# set executable path and initialize Chrome Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [5]:
#scrape page into soup
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [6]:
#scrape NASA MARS News
time.sleep(3)
latest_news_article = soup.find("div", class_='list_text')
news_title = latest_news_article.find("div", class_="content_title").text
news_p = latest_news_article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


In [23]:
#JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [24]:
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23745-640x350.jpg


In [25]:
response = requests.get(featured_image_url, stream=True)
with open('img.jpg', 'wb') as out_file:
   shutil.copyfileobj(response.raw, out_file)


from IPython.display import Image
Image(url='img.jpg')

In [26]:
#Mars Weateher

tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [27]:
#MARS Weather
tweet_url = "https://twitter.com/marswxreport?lang=en"
tweet_html_content = requests.get(tweet_url).text
soup = BeautifulSoup(tweet_html_content, "lxml")
latest_tweet = soup.find_all('div', class_="js-tweet-text-container")


#empty list to hold tweet we are going to keep, used to strip useless content from string
tweets_list = []
# Loop that scans every tweet and searches specifically for those that have weather info
for tweets in latest_tweet: 
    tweet_body = tweets.find('p').text
    if 'InSight' and 'sol' in tweet_body:
        tweets_list.append(tweet_body)
        #break statement to only print the first weather tweet found
        break
    else: 
        #if not weather related skip it and try again
        pass
    
#cleaned up tweet removes unncessary link to twitter image included in string, :-26 removes the last 26 characters which is the length of the img url
#after reviewing several links they all appear to work with the value of -26
mars_weather = ([tweets_list[0]][0][:-26])
tweet_img_link = ([tweets_list[0]][0][-26:])
print(f"{mars_weather}: {tweet_img_link}")

InSight sol 455 (2020-03-08) low -95.4ºC (-139.8ºF) high -13.0ºC (8.5ºF)
winds from the SSE at 6.0 m/s (13.5 mph) gusting to 20.7 m/s (46.2 mph)
pressure at 6.40 hPa: pic.twitter.com/HrRcPTpSEQ


In [28]:
#MARS Facts
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)



In [29]:
mars_facts_read_html = pd.read_html(mars_facts_url)
mars_facts_read_html

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [30]:
mars_facts_df = pd.DataFrame(mars_facts_read_html[0])
mars_facts_df.info

<bound method DataFrame.info of                       0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>

In [31]:
mars_facts_df.columns = ['Mars','Data']
mars_df_table= mars_facts_df.set_index("Mars")
mars_df_table

Data
Mars                                               
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers

In [32]:
#Mars facts- convert to html
mars_df_table_data = mars_df_table.to_html(classes='marsdata')
mars_df_table_data_html = mars_df_table_data.replace('\n', ' ')
mars_df_table_data_html


'<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Mars</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.39 × 10^23 kg (0.11 Earths)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.38 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-87 to -5 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

In [33]:
#MARS Hemispheres
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_hemispheres=[]

In [34]:
for i in range (4):
    time.sleep(3)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ img
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemispheres.append(dictionary)
    browser.back()

In [17]:
print(mars_hemispheres)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [18]:

mars_hemispheres_image_urls = [\
    {f'"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"'},\
    {f'"title": "Cerberus Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"'},\
    {f'"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"'},\
   {f'"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"'},\
]
mars_hemispheres_image_urls

[{'"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"'},
 {'"title": "Cerberus Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"'},
 {'"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"'},
 {'"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"'}]